In [52]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNN
import numpy as np
from functional import seq
import tensorflow as tf
from tensorpack import (TrainConfig, AsyncMultiGPUTrainer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
log_dir = 'train_log/block3-d0.5-g2-512-hidden-ngls-r5/'
logger.set_logger_dir(log_dir, action='k')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[1111 21:19:04 @logger.py:67] Existing log file 'train_log/block3-d0.5-g2-512-hidden-ngls-r5/log.log' backuped to 'train_log/block3-d0.5-g2-512-hidden-ngls-r5/log.log.1111-211904'
[1111 21:19:04 @logger.py:74] Argv: /home/yangyang/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1007/jupyter/kernel-ed3f6884-b96d-49d3-9366-9497aaaaa739.json


In [53]:
config = default
config.proportion = {'train': 0.52, 'val': 0.0, 'test': 0.48}
config.stages = [2, 3, 4, 5, 6]
config.annotation_number = 10
config.weight_decay = 0.0
config.dropout_keep_prob = 0.5
config.gamma = 2
config.use_glimpse = True
config.use_hidden = True
config.read_time = 6
config.batch_size = 64
config.max_sequence_length = 15

ignore_restore = ['learning_rate', 'global_step']
save_name = "max-micro_f1.tfmodel"
threshold = 0.4

data_manager = DataManager.from_config(config)
test_data = data_manager.get_test_stream()
train_data = data_manager.get_train_stream()
data_manager.get_num_info()

{'test': (3357, 13543), 'train': (3428, 13712), 'val': (0, 0)}

In [50]:
ignore_restore = ['learning_rate', 'global_step', ]

In [54]:
#np.set_printoptions(formatter={'float_kind': lambda x: '%.3f' % x})
model = RNN(config, label_weights=data_manager.get_imbalance_ratio().train.values)
tf.reset_default_graph()
pred_config = PredictConfig(model=model,
                            session_init=SaverRestore(
                                ignore=ignore_restore,
                                model_path=log_dir + save_name),
                            output_names=['feature', 'length', 'label'],
                            )
# pred = Predictor(pred_config, test_data, nr_proc=2, ordered=False)
pred = SimpleDatasetPredictor(pred_config, test_data)

/home/yangyang/Documents/flyexpress/DL_biomedicine_image/utils/data_manager/__init__.py:182: RuntimeWarning: invalid value encountered in double_scalars
  posi_ratio = np.sum(binary_annot, axis=0) / binary_annot.shape[0]


INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scal

In [55]:
features = []
labels = []
group_ctr = 0
img_ctr = 0
for feat_batch, length_batch, label_batch in pred.get_result():
    for i in range(0, feat_batch.shape[0]):
        group_feat = feat_batch[i, :, :]
        length = length_batch[i]
        label = label_batch[i]
        group_ctr += 1
        img_ctr += length
        '''
        for feat in group_feat[0:length]:
            features.append(feat)
            labels.append(label)
        '''
        features.append(group_feat[0:length])
        labels.append(label)

print(group_ctr)
data_set = list(zip(iter(features), iter(labels)))

100%|##########|53/53[01:51<00:00, 0.55it/s]

3357


In [24]:
import pickle

In [57]:
with open('/data/yangyang/10_labels_test_bagged.pickle', 'wb') as f:
    pickle.dump(data_set, f)

In [59]:
data_set[40][0].shape

(9, 512)